In [ ]:
import pandas as pd

In [ ]:
file=pd.read_excel("/content/Onyx Data Processed.xlsx")

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax
from tqdm.notebook import tqdm

In [ ]:
model = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model)
model_app = AutoModelForSequenceClassification.from_pretrained(model)

In [ ]:
def scores_roberta(txt):
  encoded_text = tokenizer(txt, return_tensors='pt')
  output = model(**encoded_text)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)
  scores_dict = {
      'roberta_neg' : scores[0],
      'roberta_neu' : scores[1],
      'roberta_pos' : scores[2]
  }
  return scores_dict

In [ ]:
df=df.reset_index().rename(columns={"index":"id"})

In [ ]:
res = {}
broken={}
for i, row in tqdm(df.iterrows(), total=len(df)):
    try:
        text = row["reviews.text"]
        myid = row['id']
        roberta_result = polarity_scores_roberta(text)
        res[myid]={**roberta_result}
    except RuntimeError:
        print(f'Broke for id {myid}')
        broken[myid]=text

  0%|          | 0/7081 [00:00<?, ?it/s]

Broke for id 1219
Broke for id 2112
Broke for id 2157
Broke for id 2174
Broke for id 2175
Broke for id 2258
Broke for id 2307
Broke for id 2308
Broke for id 2348
Broke for id 2384
Broke for id 2387
Broke for id 2574
Broke for id 2615
Broke for id 2873
Broke for id 2902
Broke for id 2932
Broke for id 2966
Broke for id 3376
Broke for id 3928
Broke for id 4813
Broke for id 4867
Broke for id 4886
Broke for id 4912
Broke for id 5026
Broke for id 5068
Broke for id 5080
Broke for id 5083
Broke for id 5138
Broke for id 5273
Broke for id 5337
Broke for id 5349
Broke for id 5387
Broke for id 5425
Broke for id 5431
Broke for id 5451
Broke for id 5458
Broke for id 5470
Broke for id 5475
Broke for id 5487
Broke for id 5498
Broke for id 5508
Broke for id 5517
Broke for id 5556
Broke for id 5558
Broke for id 5607
Broke for id 5617
Broke for id 5622
Broke for id 5674
Broke for id 5683
Broke for id 5704
Broke for id 5714
Broke for id 5722
Broke for id 5731
Broke for id 5741
Broke for id 5742
Broke for 

In [ ]:
broken_df = pd.DataFrame(list(broken.items()), columns=['id', 'text'])

,id,text
0,1219,The Type Cover provides a nice typing experien...
1,2112,Unlike all the good reviews I read before purc...
2,2157,I have tried out a bunch of bluetooth speakers...
3,2174,"In theory, these are a really great set of ove..."
4,2175,"In theory, these are a really great set of ove..."
...,...,...
72,6906,I used my Bose QC20i as my primary noise cance...
73,6964,The Bowers & Wilkins P7 over-the-ear headphone...
74,6995,Ultimate Ears - MEGABLAST is a stunning portab...
75,7011,"First, i want you all to know that i am a tech..."


In [ ]:
import nltk
nltk.download('punkt')

res2={}
from nltk.tokenize import sent_tokenize
def process_large_text(text):
    # Split the text into sentences
    sentences = sent_tokenize(text)

    scores = []
    for sentence in sentences:
        scores.append(polarity_scores_roberta(sentence))
    #Scores for all sentences

    # Combine scores from all sentences
    combined_scores = {}
    for score_dict in scores:
        for key, value in score_dict.items():
            if key not in combined_scores:
                combined_scores[key] = value
            else:
                combined_scores[key] += value

    # Normalize scores when needed
    total_sentences = len(sentences)
    for key in combined_scores:
        combined_scores[key] /= total_sentences

    return combined_scores

for i, row in tqdm(broken_df.iterrows(), total=len(broken_df)):
    try:
        text = row["text"]
        myid = row['id']

        roberta_result = process_large_text(text)

        res2[myid] = roberta_result
    except RuntimeError:
        print(f'Broke for id {myid}')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


  0%|          | 0/77 [00:00<?, ?it/s]

In [ ]:
#Scores for reviews where the first model didn't work because of the length of the text
res2_df=pd.DataFrame(res2).T.reset_index().rename(columns={"index":"ID"})
res2_df

,ID,roberta_neg,roberta_neu,roberta_pos
0,1219,0.120163,0.360789,0.519048
1,2112,0.478535,0.398551,0.122914
2,2157,0.196681,0.457514,0.345805
3,2174,0.215041,0.413127,0.371832
4,2175,0.211927,0.419015,0.369059
...,...,...,...,...
72,6906,0.144218,0.271239,0.584543
73,6964,0.083224,0.366020,0.550756
74,6995,0.112891,0.406398,0.480712
75,7011,0.184194,0.471262,0.344544


In [ ]:
rlsts=pd.DataFrame(res).T
rlsts=rlsts.reset_index().rename(columns={"index":"ID"})
rlsts

,ID,roberta_neg,roberta_neu,roberta_pos,sent
0,0,0.041724,0.192351,0.765925,positive
1,1,0.194587,0.758343,0.047070,neutral
2,2,0.006451,0.023775,0.969774,positive
3,3,0.003063,0.009974,0.986963,positive
4,4,0.051311,0.231020,0.717669,positive
...,...,...,...,...,...
6999,7076,0.089828,0.310266,0.599906,positive
7000,7077,0.001224,0.019332,0.979444,positive
7001,7078,0.013662,0.088240,0.898099,positive
7002,7079,0.010163,0.051725,0.938112,positive


In [ ]:
both=rlsts.append(res2_df)
both
#Putting together all the reviews with their scores

<ipython-input-202-a007d3b92fe6>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  together=rlsts.append(res2_df)


,ID,roberta_neg,roberta_neu,roberta_pos,sent
0,0,0.041724,0.192351,0.765925,positive
1,1,0.194587,0.758343,0.047070,neutral
2,2,0.006451,0.023775,0.969774,positive
3,3,0.003063,0.009974,0.986963,positive
4,4,0.051311,0.231020,0.717669,positive
...,...,...,...,...,...
72,6906,0.144218,0.271239,0.584543,NaN
73,6964,0.083224,0.366020,0.550756,NaN
74,6995,0.112891,0.406398,0.480712,NaN
75,7011,0.184194,0.471262,0.344544,NaN


In [ ]:
both["sent"]="neutral"
both.loc[(both["roberta_pos"]>both["roberta_neu"]) & (both["roberta_pos"]>both["roberta_neg"]), "sent"]="positive"
both.loc[(both["roberta_neg"]>both["roberta_neu"]) & (both["roberta_neg"]>both["roberta_pos"]), "sent"]="negative"
#Assigning categories per points

In [ ]:
both.sort_values(by="ID")

,ID,roberta_neg,roberta_neu,roberta_pos,sent
0,0,0.041724,0.192351,0.765925,positive
1,1,0.194587,0.758343,0.047070,neutral
2,2,0.006451,0.023775,0.969774,positive
3,3,0.003063,0.009974,0.986963,positive
4,4,0.051311,0.231020,0.717669,positive
...,...,...,...,...,...
6999,7076,0.089828,0.310266,0.599906,positive
7000,7077,0.001224,0.019332,0.979444,positive
7001,7078,0.013662,0.088240,0.898099,positive
7002,7079,0.010163,0.051725,0.938112,positive


In [ ]:
df.rename(columns={"id":"ID"}, inplace=True)

Index(['ID', 'brand', 'categories', 'colors', 'dimension', 'manufacturer',
       'manufacturerNumber', 'name', 'reviews.date', 'reviews.doRecommend',
       'reviews.numHelpful', 'reviews.rating', 'reviews.sourceURLs',
       'reviews.text', 'reviews.title', 'reviews.username', 'sourceURLs',
       'weight', 'categs (groups)'],
      dtype='object')

In [ ]:
final_df=both.merge(df, how="left")
#Putting the df from scores together with the rest of the columns of the dataset

In [ ]:
final_df

,ID,roberta_neg,roberta_neu,roberta_pos,sent,brand,categories,colors,dimension,manufacturer,...,reviews.doRecommend,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs,weight,categs (groups)
0,0,0.041724,0.192351,0.765925,positive,Logitech,"Electronics,Accessories & Supplies,Audio & Vid...",Black,2.2 x 1.2 x 7.3,Logitech,...,Yes,0.0,5,http://reviews.bestbuy.com/3545/3721047/review...,A lot of remotes try to be universal but with ...,This is truly Universal.,jerbo,https://www.amazon.com/Logitech-Ultimate-15-De...,1.7 pounds,Other Home & Office Electronics
1,1,0.194587,0.758343,0.047070,neutral,Logitech,"Electronics,Accessories & Supplies,Audio & Vid...",Black,2.2 x 1.2 x 7.3,Logitech,...,Yes,0.0,5,http://reviews.bestbuy.com/3545/3721047/review...,Able to hide all components in one cabinet and...,Great all-in-one remote.,DennisOKC,https://www.amazon.com/Logitech-Ultimate-15-De...,1.7 pounds,Other Home & Office Electronics
2,2,0.006451,0.023775,0.969774,positive,Logitech,"Electronics,Accessories & Supplies,Audio & Vid...",Black,2.2 x 1.2 x 7.3,Logitech,...,Yes,0.0,5,http://reviews.bestbuy.com/3545/3721047/review...,Absolutely the best remote without a doubt. Th...,Best Remote,texasman,https://www.amazon.com/Logitech-Ultimate-15-De...,1.7 pounds,Other Home & Office Electronics
3,3,0.003063,0.009974,0.986963,positive,Logitech,"Electronics,Accessories & Supplies,Audio & Vid...",Black,2.2 x 1.2 x 7.3,Logitech,...,Yes,0.0,5,http://reviews.bestbuy.com/3545/3721047/review...,added the remote to my total system and simple...,lots of attachments and works like magic,BobC,https://www.amazon.com/Logitech-Ultimate-15-De...,1.7 pounds,Other Home & Office Electronics
4,4,0.051311,0.231020,0.717669,positive,Logitech,"Electronics,Accessories & Supplies,Audio & Vid...",Black,2.2 x 1.2 x 7.3,Logitech,...,Yes,0.0,5,http://reviews.bestbuy.com/3545/3721047/review...,After owning many Harmony remotes. Logitech fi...,Logitech got it right with this remote.,Donrollo,https://www.amazon.com/Logitech-Ultimate-15-De...,1.7 pounds,Other Home & Office Electronics
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7076,6906,0.144218,0.271239,0.584543,positive,JBL,"Mobile,Headphones,Bluetooth Headphones,Electro...","Black,White",3.0 x 6.9 x 6.7,JBL,...,Yes,16.0,5,http://reviews.bestbuy.com/3545/4403709/review...,I used my Bose QC20i as my primary noise cance...,GreatSound and Value at 299.99,ryantheoc,https://www.bhphotovideo.com/c/product/1240681...,10.8 oz,Audio
7077,6964,0.083224,0.366020,0.550756,positive,Bowers & Wilkins,"Headphones,Electronics,Over-Ear Headphones,Aud...",NaN,9.2 x 7.6 x 5.2,P7,...,Yes,3.0,5,https://reviews.bestbuy.com/3545/2436009/revie...,The Bowers & Wilkins P7 over-the-ear headphone...,One of the best,bulletproofcharm,https://reviews.bestbuy.com/3545/2436009/revie...,2.43 pounds,Audio
7078,6995,0.112891,0.406398,0.480712,positive,Ultimate Ears,"Portable Bluetooth Speakers,Bluetooth & Wirele...","Blue,Graphite,Blue steel,Black,Merlot,Blizzard",3.5 x 3.5 x 9.3,984-000911,...,Yes,6.0,5,https://reviews.bestbuy.com/3545/6019903/revie...,Ultimate Ears - MEGABLAST is a stunning portab...,Audiophile's Voice-Controlled Personal Assistant,gadgetguy11,https://www.amazon.com/Ultimate-Ears-MEGABLAST...,3.8 pounds,Audio
7079,7011,0.184194,0.471262,0.344544,neutral,MEE audio,"Headphones,Bluetooth Headphones,All Headphones...","Black,Grey",NaN,HP-AF32-GW-MEE,...,Yes,128.0,5,http://reviews.bestbuy.com/3545/8826499/review...,"First, i want you all to know that i am a tech...",Best bluetooth headphones i've tried...,gemrookie,http://reviews.bestbuy.com/3545/8826499/review...,3.88 oz,Audio


In [ ]:
#Save to excel
final_df.to_excel("Onyx Data - with Sentiment Analysis.xlsx")